In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
from collections import Counter, defaultdict
import json

In [2]:
import wandb

In [3]:
!wandb login ...

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Altsoph/.netrc


In [4]:
wandb.init(project="storydb_eval.task3", reinit=True)

wandb: Currently logged in as: altsoph (use `wandb login --relogin` to force relogin)


In [5]:
work_tags = ['drama',
 'comedy',
 'television',
 'fiction',
 'series',
 'action',
 'thriller',
 'black-and-white',
 'science fiction',
 'horror']
work_langs = ['en', 'fr', 'it', 'ru', 'de', 'nl'] # , 'uk', 'pt', 'pl', 'es']

In [6]:
df = pd.read_csv('dataset3.csv')
df

,Unnamed: 0,text,labels,language,plot_id
0,0,Ed is a very opinionated 72-year-old who has b...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]",en,8517
1,1,"""Cameron James, a new student at Padua High Sc...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0]",en,410
2,2,"""Geek Hlynur is approaching 30 years old, stil...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",en,2705
3,3,"In April 2003, avid mountaineer Aron Ralston (...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",en,333
4,4,The film opens with a group of wakeboarders be...,"[0, 0, 1, 1, 0, 1, 0, 0, 0, 1]",en,10429
...,...,...,...,...,...
8623,8623,Alice Racine is een voormalige CIA-agente. Get...,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0]",nl,8844
8624,8624,Danny zit in zijn laatste jaar op de universit...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",nl,10597
8625,8625,Hoofdfiguur in de serie is een kleine jonge pi...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]",nl,5803
8626,8626,Thomas Harbor geeft een interview over een ont...,"[1, 0, 0, 1, 0, 0, 0, 0, 1, 0]",nl,6047


In [7]:
df['labels'] = df['labels'].map( lambda x: np.array(json.loads(x)) )
new_df = df.copy()
df.labels[0]

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0])

In [9]:
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModel # DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)

In [10]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
# some code is taken from https://github.com/DhavalTaunk08/NLP_scripts/blob/master/Transformers_multilabel_distilbert.ipynb

In [11]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [12]:

# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4*4
VALID_BATCH_SIZE = 4*16
EPOCHS = 10
CLASSES = 10
LEARNING_RATE = 1e-05


SAVE_PREFIX = 'mBERT_crossling'
LOG_PREFIX = 'mBERT'
MODEL = 'bert-base-multilingual-cased'

In [13]:
tokenizer = AutoTokenizer.from_pretrained(MODEL, truncation=True, do_lower_case=False)

In [14]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [15]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (8628, 5)
TRAIN Dataset: (6902, 5)
TEST Dataset: (1726, 5)


In [16]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [17]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class WorkModel(torch.nn.Module):
    def __init__(self):
        super(WorkModel, self).__init__()
        self.l1 = AutoModel.from_pretrained(MODEL)
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, CLASSES)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output


# wandb.watch(model, log="all")

In [18]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)


In [19]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets        

def valid(testing_loader, epoch=-1):
    outputs, targets = validation(testing_loader)
    final_outputs = np.array(outputs) >=0.5
    val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
    val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))
    acc = metrics.accuracy_score(targets, final_outputs)
    print(f'Acc = {acc}')
    print("AUROC per tag")
    wandb.log({"hamming_score": val_hamming_score, "hamming_loss": val_hamming_loss,
              "acc": metrics.accuracy_score(targets, final_outputs), 'epoch':epoch,
              'model': LOG_PREFIX})
    for i, name in enumerate(work_tags):
        try:
            tag_auroc = metrics.roc_auc_score(np.array(targets)[:, i], final_outputs[:, i]) # , pos_label=1)
            wandb.log({f"tag_aucroc/{name}": metrics.roc_auc_score(np.array(targets)[:, i], final_outputs[:, i]),
                   f"tag_acc/{name}": metrics.accuracy_score(np.array(targets)[:, i], final_outputs[:, i]), 'epoch':epoch,
                       'model': LOG_PREFIX
                  })
        except:
            pass
#     for cl in work_langs:
#         lang_test_data = test_data[test_data['language']==cl]
#         lang_test_data = lang_test_data.reset_index(drop=True)
#         lang_testing_set = MultiLabelDataset(lang_test_data, tokenizer, MAX_LEN)
#         lang_testing_loader = DataLoader(lang_testing_set, **test_params)
#         lang_outputs, lang_targets = validation(lang_testing_loader)
#         lang_final_outputs = np.array(lang_outputs) >=0.5
#         wandb.log({f"lang_acc/{cl}": metrics.accuracy_score(lang_targets, lang_final_outputs), 'epoch':epoch, 'model': LOG_PREFIX})
    return acc

def valid_cross(nl, epoch=-1):
    for cl in work_langs:
        lang_test_data = new_df.copy()
        lang_test_data = lang_test_data[lang_test_data['language']==cl]
        lang_test_data = lang_test_data.reset_index(drop=True)
        lang_testing_set = MultiLabelDataset(lang_test_data, tokenizer, MAX_LEN)
        lang_testing_loader = DataLoader(lang_testing_set, **test_params)
        lang_outputs, lang_targets = validation(lang_testing_loader)
        lang_final_outputs = np.array(lang_outputs) >=0.5
        wandb.log({f"lang_acc/{cl}": metrics.accuracy_score(lang_targets, lang_final_outputs),'model': LOG_PREFIX, 'epoch':epoch})
    return acc

In [20]:
for cl in work_langs:
    model = WorkModel()
    model.to(device)
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

    lang_data = new_df[new_df['language']==cl]
    lang_train_data = lang_data.sample(frac=train_size,random_state=200)
    lang_test_data = lang_data.drop(lang_train_data.index).reset_index(drop=True)
    lang_train_data = lang_train_data.reset_index(drop=True)

    training_set = MultiLabelDataset(lang_train_data, tokenizer, MAX_LEN)
    testing_set = MultiLabelDataset(lang_test_data, tokenizer, MAX_LEN)

    prev_acc = -1
    
    training_loader = DataLoader(training_set, **train_params)
    testing_loader = DataLoader(testing_set, **test_params)    

    run = wandb.init(project="storydb_eval.task3", reinit=True)
    wandb.run.name = f'{LOG_PREFIX}_{cl}'
    wandb.run.save()    
    
    best_model = None
    output_model_file = None
    for epoch in range(EPOCHS):
        acc = valid(testing_loader, epoch=epoch)
        valid_cross(cl, epoch = epoch)
        if acc<prev_acc and best_model: break
        prev_acc = acc
        best_model = output_model_file
        train(epoch)
        output_model_file = f'./model_{SAVE_PREFIX}_lang_{cl}_ep{epoch+1}.bin'
        torch.save(model.state_dict(), output_model_file)
    if acc>prev_acc:
        acc = valid(testing_loader, epoch=EPOCHS)
        if acc>prev_acc:
            best_model = output_model_file
    
    print(f'loading best model, it was {best_model}')
    model.load_state_dict(torch.load(best_model))
    
    run.finish()


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
5it [00:02,  1.92it/s]
0it [00:00, ?it/s]

Acc = 0.0
AUROC per tag


23it [00:10,  2.11it/s]
23it [00:08,  2.61it/s]
23it [00:10,  2.28it/s]
23it [00:10,  2.21it/s]
23it [00:10,  2.16it/s]
23it [00:09,  2.54it/s]
0it [00:00, ?it/s]

Epoch: 0, Loss:  0.7088102698326111


72it [00:26,  2.68it/s]
5it [00:02,  2.25it/s]
0it [00:00, ?it/s]

...

Acc = 0.3159722222222222
AUROC per tag


23it [00:11,  2.06it/s]
23it [00:09,  2.54it/s]
23it [00:10,  2.19it/s]
23it [00:10,  2.15it/s]
23it [00:10,  2.11it/s]
23it [00:09,  2.49it/s]


loading best model, it was ./model_mBERT_crossling_lang_nl_ep6.bin


hamming_score,0.43299
hamming_loss,0.13264
acc,0.31597
epoch,7
model,mBERT
_runtime,730
_timestamp,1626619105
_step,135
tag_aucroc/drama,0.70924
tag_acc/drama,0.75
tag_aucroc/comedy,0.63802


hamming_score,▃▁▄▆▇███
hamming_loss,█▁▁▁▁▁▁▁
acc,▁▁▅▆▇███
epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
_runtime,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇██
_timestamp,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇██
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
tag_aucroc/drama,▂▁▆▇█▇▇▇
tag_acc/drama,▁▆▆▇▇███
tag_aucroc/comedy,▁▁▁▃▆██▆
tag_acc/comedy,▁▁▁▄█▆▅▇
